In [1]:
#dependencies 
import pandas as pd

In [2]:
#load files
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

In [3]:
#read data files and store them into Pandas data frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [4]:
#merge into a single data set
school_data_complete = pd.merge(student_data, school_data, how = "left", on = ["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
total_schools = school_data["school_name"].count()
total_students = student_data["student_name"].count()
total_budget = school_data["budget"].sum()
average_math_score = student_data["math_score"].mean()
average_reading_score = student_data["reading_score"].mean()
overall_average = ((average_math_score + average_reading_score)/2)
passing_math = ((student_data["math_score"] >= 70).mean())*100
passing_reading = ((student_data["reading_score"] >= 70).mean())*100


In [6]:
#Create Disctrict Summary Data Frame
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [average_math_score],
                                   "Average Reading Score": [average_reading_score],
                                   "% Passing Math": [passing_math],
                                   "% Passing Reading": [passing_reading],
                                   "Overall Average Score": [overall_average]})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Average Score
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


In [7]:
#add passing math and reading to school_data_complete
school_data_complete["PassingMath"] = school_data_complete["math_score"] >= 70
school_data_complete["PassingReading"] = school_data_complete["reading_score"] >= 70


In [8]:
#group school_data_complete by school name
grouped_schools = school_data_complete.groupby(["school_name"]).mean()


In [9]:
#make into percentage
grouped_schools["% Passing Math"] = grouped_schools["PassingMath"]*100
grouped_schools["% Passing Reading"] = grouped_schools["PassingReading"]*100

In [10]:
#merge grouped_schools y with school_data
schoolData_summary = pd.merge(grouped_schools, school_data, how = "left", on = ["school_name", "school_name"])
schoolData_summary.head()

,school_name,Student ID,reading_score,math_score,School ID_x,size_x,budget_x,PassingMath,PassingReading,% Passing Math,% Passing Reading,School ID_y,type,size_y,budget_y
0,Bailey High School,20358.5,81.033963,77.048432,7.0,4976.0,3124928.0,0.666801,0.819333,66.680064,81.933280,7,District,4976,3124928
1,Cabrera High School,16941.5,83.975780,83.061895,6.0,1858.0,1081356.0,0.941335,0.970398,94.133477,97.039828,6,Charter,1858,1081356
2,Figueroa High School,4391.0,81.158020,76.711767,1.0,2949.0,1884411.0,0.659885,0.807392,65.988471,80.739234,1,District,2949,1884411
3,Ford High School,36165.0,80.746258,77.102592,13.0,2739.0,1763916.0,0.683096,0.792990,68.309602,79.299014,13,District,2739,1763916
4,Griffin High School,12995.5,83.816757,83.351499,4.0,1468.0,917500.0,0.933924,0.971390,93.392371,97.138965,4,Charter,1468,917500


In [11]:
#calculate per student budget and the overall passing rate
per_student_budget = (schoolData_summary['budget_x']/schoolData_summary['size_x'])
overall_passing_rate = (schoolData_summary['% Passing Math'] + schoolData_summary['% Passing Reading'])/2

In [12]:
#Create a School Summary Data Frame
school_summary_table = pd.DataFrame({"School Name": schoolData_summary['school_name'], 
                                     "Type": schoolData_summary['type'],
                                    "Total Students": schoolData_summary['size_x'],
                                    "Total School Budget": schoolData_summary['budget_x'],
                                    "Per Student Budget": per_student_budget,
                                   "Average Math Score": schoolData_summary['math_score'],
                                   "Average Reading Score": schoolData_summary['reading_score'],
                                    "% Passing Math": schoolData_summary['% Passing Math'],
                                    "% Passing Reading": schoolData_summary['% Passing Reading'],
                                    "Overall Passing Rate": overall_passing_rate})
school_summary_table.head()

,School Name,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


In [13]:
#top performing schools by passing rate
top_performing = school_summary_table.sort_values("Overall Passing Rate", ascending = False)
top_performing.head()

,School Name,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
12,Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
13,Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [14]:
#bottom performing schools by passing rate
bottom_performing = school_summary_table.sort_values("Overall Passing Rate")
bottom_performing.head()

,School Name,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
10,Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
2,Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
7,Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
3,Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [15]:
#Create pandas series for for 9, 10, 11, 12 grade and group by school name
nineth_grade = school_data_complete[school_data_complete["grade"] == "9th"].groupby(["school_name"]).mean()
tenth_grade = school_data_complete[school_data_complete["grade"] == "10th"].groupby(["school_name"]).mean()
eleventh_grade = school_data_complete[school_data_complete["grade"] == "11th"].groupby(["school_name"]).mean()
twelveth_grade = school_data_complete[school_data_complete["grade"] == "12th"].groupby(["school_name"]).mean()

In [16]:
#Math Scores By Grade
math_scores_by_grade_df = pd.DataFrame({"9th": nineth_grade['math_score'],
                                       "10th": tenth_grade['math_score'],
                                       "11th": eleventh_grade['math_score'],
                                       "12th": twelveth_grade['math_score']})
math_scores_by_grade_df.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


In [17]:
#Reading Scores By Grade
reading_score_by_grade_df = pd.DataFrame({"9th": nineth_grade['reading_score'],
                                       "10th": tenth_grade['reading_score'],
                                       "11th": eleventh_grade['reading_score'],
                                       "12th": twelveth_grade['reading_score']})
reading_score_by_grade_df.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


In [18]:
# Sample bins. Feel free to create your own bins.
#Scores By School Spending
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [21]:
school_summary_table["Spending Ranges"] = pd.cut(school_summary_table["Per Student Budget"], spending_bins, labels = group_names)

In [48]:
scores_by_spending = school_summary_table.groupby("Spending Ranges").mean()
scores_by_spending

,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges,,,,,,,,
<$585,1592.000000,9.246042e+05,581.000000,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,1361.500000,8.212290e+05,604.500000,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,2961.000000,1.880208e+06,635.166667,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,4104.333333,2.675768e+06,652.333333,76.997210,81.027843,66.164813,81.133951,73.649382


In [53]:
#delete extra columns
del scores_by_spending["Total Students"]
del scores_by_spending["Total School Budget"]
del scores_by_spending["Per Student Budget"]

In [54]:
#display scores by school spending
scores_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [49]:
#Scores By School Size 
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [50]:
school_summary_table["Size Ranges"] = pd.cut(school_summary_table["Total Students"], size_bins, labels=group_names)

In [55]:
scores_by_size = school_summary_table.groupby("Size Ranges").mean()
scores_by_size

,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Size Ranges,,,,,,,,
Small (<1000),694.500,416972.500,595.000,83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),1704.400,1029597.200,605.600,83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),3657.375,2333437.125,635.375,77.746417,81.344493,69.963361,82.766634,76.364998


In [56]:
#delete extra columns
del scores_by_size["Total Students"]
del scores_by_size["Total School Budget"]
del scores_by_size["Per Student Budget"]

In [57]:
#display scores by school size
scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Size Ranges,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [59]:
scores_by_type = school_summary_table.groupby("Type").mean()
scores_by_type

,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,,,,
Charter,1524.250000,9.126881e+05,599.500000,83.473852,83.896421,93.620830,96.586489,95.103660
District,3853.714286,2.478275e+06,643.571429,76.956733,80.966636,66.548453,80.799062,73.673757


In [60]:
#delete extra columns
del scores_by_type["Total Students"]
del scores_by_type["Total School Budget"]
del scores_by_type["Per Student Budget"]

In [61]:
#display scores by school type
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [ ]:
#Observable Trends

#(1)Charter Schools has better students than District schools since the Overall Passing Rate is 95.10 to 73.67 respectively 
#(2)I found very interisting that schools that spend less have a better passing rate than those schools with a higher spending rate